<a href="https://colab.research.google.com/github/Maxvgrad/Binary-Classification-with-a-Software-Defects-Dataset/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download dataset

In [2]:
from google.colab import files

files.upload(); # upload your kaggle.json file

Saving kaggle.json to kaggle.json


In [3]:
import json

!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

mkdir: cannot create directory ‘/root/.kaggle/’: File exists
- path is now set to: {/content}


In [4]:
dataset_name = 'playground-series-s3e23'
previous_dataset_path = '/content/{/content}/competitions/' + dataset_name
dataset_path = previous_dataset_path.replace('/{/content}', '')
dataset_archive = dataset_name + '.zip'

In [5]:
!kaggle competitions download -c {dataset_name}

  0% 0.00/5.71M [00:00<?, ?B/s]
100% 5.71M/5.71M [00:00<00:00, 63.7MB/s]


In [6]:
!mkdir -p competitions/playground-series-s3e23
!mv {previous_dataset_path}/{dataset_archive} {dataset_path}/{dataset_archive}
!unzip {dataset_path}/{dataset_archive} -d {dataset_path}

# Optional clean up
!rm -r '{'
!rm {dataset_path}/{dataset_archive}

Archive:  /content/competitions/playground-series-s3e23/playground-series-s3e23.zip
  inflating: /content/competitions/playground-series-s3e23/sample_submission.csv  
  inflating: /content/competitions/playground-series-s3e23/test.csv  
  inflating: /content/competitions/playground-series-s3e23/train.csv  


## Base line submission

In [7]:
!kaggle competitions submit -c {dataset_name} -f {dataset_path}/sample_submission.csv -m "Sample submission test"

100% 729k/729k [00:00<00:00, 2.56MB/s]
400 - Bad Request - Submission not allowed:  Submissions have been disabled for this competition.
